In [3]:
import pandas as pd
import missingno as msno
from genesis import *

from datetime import date
DATE_RANGE = pd.date_range(date(2000,1,3), date(2022,1,1), freq='B')

In [4]:
def fill_date(data, fill_forward = False):
  date_range_df = pd.DataFrame(DATE_RANGE).set_index(0)
  date_range_df.index.names = ["Dates"]
  data.index.names = ["Dates"]
  if fill_forward:
    filled_data = pd.merge(date_range_df, data, how="outer", on="Dates").ffill()
  else:
    filled_data = pd.merge(date_range_df, data, how="outer", on="Dates")
  return filled_data

def standardize_series(adf, typ):
  df = adf.copy()
  problems = [list(df).index(x) for x in list(df) if x[:3] != "USD"]
  tmp_names = list(map(lambda x: df.columns[x] if x not in problems else (df.columns[x][3:] + df.columns[x][:3]) , range(len(df.columns))))
  for i in problems:
    if typ == "prices":
      df.iloc[:,i] = 1/df.iloc[:,i]
    elif typ == "returns":
      df.iloc[:,i] = df.iloc[:,i] * -1
  df.columns = tmp_names
  return df

In [5]:
raw_data = pd.read_csv("../Dymon/Code Data/NUS_Data.csv", index_col=0, header=1)

c:\users\vin_s\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
fx = raw_data[[x for x in list(raw_data) if ("USD" in x or "+1" in x)]]
spot_ndf = fx[[x for x in fx.columns if fx.loc["field", x] == "PX_LAST"]]
spot_ndf = spot_ndf.iloc[2:,:]
spot_ndf.columns = ["EURUSD", "GBPUSD", "AUDUSD", "NZDUSD", "USDJPY", "USDCAD", "USDSEK", "USDNOK", 
                    "USDCNY", "USDCNH", "USDSGD", "USDCHF", "USDTHB", "USDIDR", "USDINR", "USDMYR",
                    "USDTWD", "USDKRW", "USDPHP"]
spot_ndf.index.name = "date"
spot_ndf.index = pd.to_datetime(spot_ndf.index)
spot_ndf = spot_ndf.astype(float)
spot = spot_ndf.copy()

fp = raw_data[[x for x in list(raw_data) if x in ["IHN1M CMPN Curncy", "IRN1M CMPN Curncy", "MRN1M CMPN Curncy", "NTN1M CMPN Curncy", "KWN1M CMPN Curncy", "PPN1M CMPN Curncy"]]].iloc[2:]
fp.columns = ["USDIDR", "USDINR", "USDMYR", "USDTWD", "USDKRW", "USDPHP"]
fp = fp.astype(float)
fp.index = pd.to_datetime(fp.index)

spot["USDIDR"] = spot_ndf["USDIDR"] - fp["USDIDR"]
spot["USDINR"] = spot_ndf["USDINR"] - fp["USDINR"]/100
spot["USDMYR"] = spot_ndf["USDMYR"] - fp["USDMYR"]/10000
spot["USDTWD"] = spot_ndf["USDTWD"] - fp["USDTWD"]
spot["USDKRW"] = spot_ndf["USDKRW"] - fp["USDKRW"]
spot["USDPHP"] = spot_ndf["USDPHP"] - fp["USDPHP"]

In [7]:
px = read_prices("../Dymon/Code Data/carry_adj_fx_returns.csv")
px.columns = [x[0] for x in px.columns]

In [8]:
total_price_ori = px.pipe(fill_date).pipe(omit_trailing_na).pipe(omit_leading_na, how = "any").ffill(limit = 2)
total_price_usd = total_price_ori.pipe(standardize_series, "prices")
total_returns_ori = total_price_ori.pct_change(fill_method=None)
total_returns_usd = total_returns_ori.pipe(standardize_series, "returns")

In [9]:
spot.loc[:"2005", ["USDINR", "USDMYR", "USDKRW", "USDPHP"]] = np.nan
spot.loc[:"2007-06-30", ["USDIDR"]] = np.nan

In [10]:
spot_price_ori = spot.pipe(fill_date).pipe(omit_trailing_na).pipe(omit_leading_na, how = "any").ffill(limit = 2)
spot_price_usd = spot_price_ori.pipe(standardize_series, "prices")
spot_returns_ori = spot_price_ori.pct_change(fill_method=None)
spot_returns_usd = spot_returns_ori.pipe(standardize_series, "returns")

In [11]:
G10_FX = ["USDAUD", "USDCAD", "USDEUR", "USDGBP", "USDJPY", "USDNZD", "USDSEK", "USDNOK", "USDCHF"]
ASIA_FX = ['USDCNH', 'USDSGD', 'USDTHB', 'USDIDR', 'USDINR', 'USDMYR', 'USDTWD', 'USDKRW', 'USDPHP']
ALL_FX = G10_FX + ASIA_FX

In [ ]:
carry_returns_ori = total_returns_ori - spot_returns_ori
carry_returns_usd = total_returns_usd - spot_returns_usd